In [ ]:
%pip install langchain-ollama pydantic langsmith python-dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from pydantic import BaseModel, Field

class Response(BaseModel):
    question: str = Field(description="The question asked.")
    answer: str = Field(description="The answer to the question.")
    reasoning: str = Field(description="The reasoning behind the answer.")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

system_prompt = """You are a helpful assistant. You will be given a question and you need to answer it step by step."""

template = """Question: {question}

Answer: Let's think step by step."""

parser = StrOutputParser()


prompt = ChatPromptTemplate.from_messages([
  ("system", system_prompt),
  ("human", template),
])

model = ChatOllama(model="deepseek-r1:7b", base_url="http://localhost:11434")
# .with_structured_output(Response)


In [ ]:
from langchain_core.runnables import RunnableLambda

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: len(x.split()))

In [ ]:
chain = prompt | model | parser | uppercase_output | count_words
response = chain.invoke({"question": "What is the capital of Thailand?"})
response

In [ ]:
promtpt_template = ChatPromptTemplate.from_messages([
  ("system", "You are an expert product reviewer."),
  ("human", "List the main features of the product {product_name}."),
])

pros_template = ChatPromptTemplate.from_messages([
  ("system", "You are an expert product reviewer."),
  ("human", "Given these features: {features}, list the pros of these features."),
])


cons_template = ChatPromptTemplate.from_messages([
  ("system", "You are an expert product reviewer."),
  ("human", "Given these features: {features}, list the cons of these features."),
])

pros_chain = pros_template | model | parser
cons_chain = cons_template | model | parser

output_parser = RunnableLambda(lambda x: f"Pros: {x['pros']}, Cons: {x['cons']}")

product_chain = promtpt_template | model | parser | {
  "pros": pros_chain,
  "cons": cons_chain,
} | output_parser

product_chain.invoke({"product_name": "iPhone 14 Pro Max"})



In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableMap

hello_runnable = RunnableLambda(lambda x: f"Hello {x['wording']['other']}")

chain = RunnableMap({
    "question": RunnableLambda(lambda x: "test"),
    "wording": RunnablePassthrough(),
}) | hello_runnable

response = chain.invoke({"wording": "World", "other": "test"})
print(response)